<h1>Detection using Transfer Learning</h1>

<h3>Using Keras</h3>

Model created using Google's <a href="https://teachablemachine.withgoogle.com/">Teachable Machine Learning</a>

In [5]:
#import libraries
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import os

In [6]:
def detect(file):
    # Load the model
    model = load_model('lensing.h5', compile=False)
    # Load the labels
    class_names = open('labels.txt', 'r').readlines()

    # Create the array of the right shape to feed into the keras model
    # The 'length' or number of images you can put into the array is
    # determined by the first position in the shape tuple, in this case 1.
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    image = Image.open(file).convert('RGB')

    #resize the image to a 224x224 with the same strategy as in TM2:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size)

    #turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array
    
    # run the inference
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]
    return (class_name,confidence_score)

In [7]:
#sample output
category,confidence = detect('test/lensed/lensed.jpg')
print('Image is:', category, end='')
print('Confidence score:', confidence*100,'%')

Image is: Lensed
Confidence score: 99.75656270980835 %


In [8]:
test_lensed = os.listdir('test/lensed/')
test_unlensed = os.listdir('test/unlensed/')

lensed_result = []
unlensed_result = []
for i in test_lensed:
    path = 'test/lensed/'+i
    category,confidence = detect(path)
    lensed_result.append(category[:-1])
for i in test_unlensed:
    path = 'test/unlensed/'+i
    category,confidence = detect(path)
    unlensed_result.append(category[:-1])

In [9]:
lensed = []
unlensed = []
for i in lensed_result:
    if i == 'Lensed':
        lensed.append(1)
    else:
        lensed.append(0)
for i in unlensed_result:
    if i == 'Unlensed':
        unlensed.append(1)
    else:
        unlensed.append(0)

In [20]:
pred = lensed+unlensed
true = []
for i in range(len(pred)):
    true.append(1)
print(pred,true)

[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [22]:
import keras
metric = keras.metrics.Accuracy()
metric.update_state(true,pred)
print("Accuracy: ",metric.result().numpy()*100,"%")

Accuracy:  70.58823704719543 %
